In [1]:
import requests
import json
import time
from typing import Optional, Dict, Any

In [2]:
class LangGraphClient:
    def __init__(self, base_url: str, api_key: str):
        """
        Client pour interagir avec LangGraph Cloud API
        
        Args:
            base_url: URL de base de l'API (ex: https://your-deployment.langchain.com)
            api_key: Clé API pour l'authentification
        """
        self.base_url = base_url.rstrip('/')
        self.headers = {
            'Authorization': f'Bearer {api_key}',
            'Content-Type': 'application/json'
        }
    
    def create_thread(self) -> str:
        """Créer un nouveau thread de conversation"""
        response = requests.post(
            f"{self.base_url}/threads",
            headers=self.headers,
            json={}
        )
        response.raise_for_status()
        return response.json()['thread_id']
    
    def start_run(self, thread_id: str, assistant_id: str, input_data: Dict[str, Any]) -> str:
        """
        Démarrer un run en streaming
        
        Args:
            thread_id: ID du thread
            assistant_id: ID de votre graph déployé (ex: "campaign-assistant")
            input_data: Données d'entrée pour le graph
        """
        url = f"{self.base_url}/threads/{thread_id}/runs/stream"
        
        payload = {
            "assistant_id": assistant_id,
            "input": input_data,
            "stream_mode": ["values", "messages", "events"]
        }
        
        response = requests.post(url, headers=self.headers, json=payload, stream=True)
        response.raise_for_status()
        
        return response
    
    def get_run_status(self, thread_id: str, run_id: str) -> Dict[str, Any]:
        """Récupérer le statut d'un run"""
        response = requests.get(
            f"{self.base_url}/threads/{thread_id}/runs/{run_id}",
            headers=self.headers
        )
        response.raise_for_status()
        return response.json()
    
    def resume_run(self, thread_id: str, run_id: str, user_input: Any) -> requests.Response:
        """
        Reprendre un run interrompu avec l'input utilisateur
        
        Args:
            thread_id: ID du thread
            run_id: ID du run à reprendre
            user_input: Input utilisateur (peut être string, dict, etc.)
        """
        url = f"{self.base_url}/threads/{thread_id}/runs/{run_id}/stream"
        
        # Construire le payload selon votre structure MyState
        payload = {
            "input": user_input,
            "stream_mode": ["values", "messages", "events"]
        }
        
        response = requests.post(url, headers=self.headers, json=payload, stream=True)
        response.raise_for_status()
        
        return response

In [3]:
client = LangGraphClient(
        base_url="http://0.0.0.0:8000",  # Remplacer par votre URL
        api_key="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"  # Remplacer par votre clé API
    )